# [Google Colab] - Modelo RAG em Documento sobre Tabagismo

Notebook usado para desenvolvimento de aplicação que utiliza a técnica de Retrieval Augmented Generation (RAG) através dos dados médicos de protocolo de tratamento do tabagismo.

**Este Notebook foi utilizado dentro do Google Colab, caso for utilizado localmente algumas mudanças devem ser necessárias.**

## 1 - Leitura dos Dados

A primeira etapa para o desenvolvimento é a leitura dos dados do PDF.

In [ ]:
# Instalação de bibliotecas necessárias para o funcionamento do notebook
! pip install unstructured[pdf] pytesseract langchain langchain_community fastembed chromadb
! apt install tesseract-ocr
! apt install tesseract-ocr-por
! apt install libtesseract-dev
! apt-get install poppler-utils

Download do PDF através do Link do site do INCA.


In [ ]:
import requests

FILENAME = 'protocolo-clinico-e-diretrizes-terapeuticas-do-tabagismo.pdf'

url = 'https://www.inca.gov.br/sites/ufu.sti.inca.local/files//media/document//'
response = requests.get(url + FILENAME, stream=True)

with open(FILENAME, 'wb') as f:
    f.write(response.content)

Leitura do PDF utilizando o tesseract-ocr. Esta biblioteca utiliza modelos de IA para fazer a leitura do PDF e conseguir agrupar as informações mais corretamente, diferenciando os tipos de dados encontrados, como texto, tabelas, etc.

In [ ]:
from unstructured.partition.pdf import partition_pdf
import pytesseract

result_pdf = partition_pdf(filename=FILENAME, infer_table_structure=True,
                           strategy="hi_res", chunking_strategy="by_title",
                           languages=["por"])

Importação dos dados do PDF para json, caso queira já usar os dados extraidos sem ter que fazer a leitura através do tesseract.

In [ ]:
import json

pdf_to_text = {}
pdf_to_text['text'] = [str(r) for r in result_pdf]
with open('pdf_to_text.json', 'w', encoding='utf-8') as f:
    json.dump(pdf_to_text, f, ensure_ascii=False, indent=4)

Tipos de dados utilizados pelo modelo Llama.

In [ ]:
from langchain.storage import InMemoryStore
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.retrievers.multi_vector import MultiVectorRetriever

# InMemoryStore é área de armazenamento que manterá os documentos
docstore = InMemoryStore()
id_key = "tabagismo_id"

# Embedding (https://github.com/qdrant/fastembed/)
embedding = FastEmbedEmbeddings()

# Banco chroma que irá manter os vetores de informação
vectorstore = Chroma(collection_name="summaries11",
                     embedding_function=embedding)

# Definição do MultiVectorRetriever que irá organizar os
# dados do chroma e as informações dos documentos
retriever = MultiVectorRetriever(vectorstore=vectorstore,
                                 docstore=docstore,
                                 id_key=id_key)

Informação retirada do PDF sendo adicionada na formatação a ser utilizada pelo modelo de RAG.

In [ ]:
from langchain_core.documents import Document
from unstructured.documents.elements import CompositeElement, Table, TableChunk

# Cria duas listas contendo:
# 1 - IDs dos documentos
# 2 - Documento no formato para a LLM, contendo a informação em formato str
docs_id, docs = [], []
for i, info in enumerate(result_pdf):
    docs_id += [str(i)]
    docs += [Document(page_content=str(info), metadata={id_key: str(i)})]

# Adiciona os documentos no MultiVectorRetriever que será utilizado
# para inserir os dados de contexto
retriever.vectorstore.add_documents(docs)
retriever.docstore.mset(list(zip(docs_id, docs)))

## 2 - Inicialização do Modelo llama3

Com os dados disponíveis e organizados de forma que o modelo possa ser utilizado, agora é necessário inicializar o modelo dentro do Colab para testar seu funcionamento.

Modelo llama3 utilizando o Prompt com as regras a serem seguidas.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """
Context: Você é um assistente especialista no Protocolo Clínico e diretrizes Terapeuticas do Tabagismo.\
Os dados que você tem disponível se trata de um documento disponibilizado pelo INCA (Instituto Nacional do Câncer).\
Todo questionamento respondido deve utilizar como base os dados passados no texto abaixo, caso não seja possível\
solucionar a dúvida, informe que não possuí informações disponíveis. Aqui está o texto disponível: \
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
model = Ollama(model="llama3")

chain = ({"context": retriever, "question": RunnablePassthrough()}
         | prompt | model | StrOutputParser())

Para conseguir rodar o modelo do llama3, é preciso "criar" um terminal dentro do colab para deixar o serviço rodando. Para isso, é preciso inicializar a célula abaixo e adicionar as seguintes linhas quando o terminal estiver disponível:

```
curl -fsSL https://ollama.com/install.sh | sh
ollama pull llama3 & ollama serve
```



In [ ]:
!pip install colab-xterm
%load_ext colabxterm
%xterm

Espere o modelo terminar de ser baixado e inicializar antes de seguir nas próximas células. Caso não tenha inicializado ainda, irá ocorrer um erro. Então, apenas espere finalizar e o resto funcionará corretamente.

## 3 - Questione o modelo!

Com o modelo treinado e funcionando, podemos inicializar seu questionamento. Abaixo temos algumas perguntas já criadas para ver o seu funcionamento. Além disso, na última célula você pode alterar para criar as suas próprias perguntas! Divirta-se!

In [ ]:
# Método que recebe a pergunta, faz para o modelo e retorna sua resposta.
def ask_question(question):
    answer = chain.invoke(question)
    print(answer)

In [ ]:
QUESTION = 'Fumar é muito prejudicial para o ser humano?'

ask_question(QUESTION)

In [ ]:
QUESTION = 'Como saber se estou dependente da nicotina? Existe alguma forma de avaliar o grau de depêndencia da pessoa?'

ask_question(QUESTION)

In [ ]:
QUESTION = 'Quais os medicamentos indicados para tentar parar de fumar?'

ask_question(QUESTION)

In [ ]:
QUESTION = 'Você poderia me dizer algo sobre a empresa A3Data?'

ask_question(QUESTION)

In [ ]:
QUESTION = 'INSIRA SUA PERGUNTA'

ask_question(QUESTION)